# Heatmap with UNIX Timestamp #
# Creates df_points.csv with all points and idletimes #

### ca 1h for 1 year

In [45]:
def importdata():

    # show all rows
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)

    # import Data
    df01 = pd.read_csv('/Users/jonweske/devment/BA_nextbike/data/lendings_2019.csv')


    df01 = df01[['bike_id', 'start_time', 'end_time', 'start_lat', 'start_lng', 'end_lat', 'end_lng']]

    # cut
    # df01 = df01.truncate(after=10000)
    return df01

In [46]:
def dicOutOfDf(df01):
    import pandas as pd
    from tqdm import tqdm
    # Dict out of DataFrames of Trips per Bike

    # init DFs for every bike
    myDFs = {}

    for ind in tqdm(df01.index):
        if df01['bike_id'][ind] not in myDFs:
            myDFs[df01['bike_id'][ind]] = pd.DataFrame(columns=
                                                       ['bike_id', 'start_time', 'end_time', 'start_lat', 'start_lng', 'end_lat', 'end_lng'])

    return myDFs

In [47]:
def addRentalsToDf(df01,myDFs):
    from tqdm import tqdm
    # Add all rentals to their bike_id DF
    for ind in tqdm(df01.index):
        if df01['bike_id'][ind] in myDFs:
            myDFs[df01['bike_id'][ind]].loc[df01.index[ind]] = df01.iloc[ind]
    return myDFs

In [48]:
def sortDFs(myDFs):
    # sort every DF
    for df in myDFs.values():
        df[['bike_id', 'start_time','end_time']] = df[['bike_id', 'start_time','end_time']].astype(int)
        df[['start_lat','start_lng', 'end_lat', 'end_lng']] = df[['start_lat','start_lng', 'end_lat', 'end_lng']].astype(float)
        df.sort_values(by=['start_time'], inplace=True)
        df.reset_index(drop=True, inplace=True)
    return myDFs

In [49]:
def dfPoints(myDFs):
    # DF of all points with stoodtime
    import pandas as pd
    from tqdm import tqdm
    df_points = pd.DataFrame(columns=['bike_id', 'lat', 'lng', 'timestood', 'time_start', 'time_end'])
    df_points[['bike_id', 'timestood', 'time_start', 'time_end']] = df_points[['bike_id', 'timestood', 'time_start', 'time_end']].astype(int)
    df_points[['lng', 'lat']] = df_points[['lng', 'lat']].astype(float)

    for df in tqdm(myDFs.values()):
        df = df.reset_index(drop=True)
        for ind in df.index[1:]:
            try:
                timestood = ((df['start_time'][ind] - df['end_time'][ind - 1]) // 60 ).astype(int)
            except:
                print("Timestood = 0 ----- " + str(df['start_time'][ind]) + "  -  " + str(df['end_time'][ind - 1]))
                timestood = 0
            dict = {'bike_id': df['bike_id'][ind], 'lng': df['start_lng'][ind], 'lat': df['start_lat'][ind],
                    'timestood': timestood, 'time_start': df['end_time'][ind - 1], 'time_end': df['start_time'][ind]}
            invalidGeo(df['start_lat'][ind-1], df['start_lng'][ind-1], df['start_lat'][ind], df['start_lng'][ind-1])
            df_points = df_points.append(dict, ignore_index=True)
    return df_points

In [50]:
def invalidGeo(Alat, Alng, Blat, Blng):
    a = (Alat, Alng) 
    b = (Blat, Blng)
    print(str(geodesic(a, b).m) + "   ///    " + str(Alat) + " to "+ str(Blat) + " /// " + str(Alng) + " to "+ str(Blng))

In [51]:
def keplerMap(df_points):
    heatmap = KeplerGl()
    heatmap.add_data(data=df_points, name='points')
    heatmap.save_to_html(file_name='heatmap_test.html')

In [52]:
from keplergl import KeplerGl
import pandas as pd
pd.options.mode.chained_assignment = None
from tqdm import tqdm
from geopy.distance import geodesic 
df = pd.DataFrame()

#df = getDatafromDB()
df = importdata()
myDfs = dicOutOfDf(df)
myDfs = addRentalsToDf(df, myDfs)
myDfs = sortDFs(myDfs)

#print(myDfs[73817])
#df_points = dfPoints(myDfs)

for ind in myDfs[73817].index[1:]:
    print(df['start_time'][ind]>df['end_time'][ind-1])
    #invalidGeo(df['start_lat'][ind-1], df['start_lng'][ind-1], df['start_lat'][ind], df['start_lng'][ind-1])

#print max min
#print(df_points['timestood'].max())
#print(df_points['timestood'].min())

# save CSV and Map
df_points.to_csv('df_points.csv', index=False)
keplerMap(df_points)


100%|███████████████████████████████████████| 604598/604598 [13:28<00:00, 747.65it/s]


1
1477.1349922448585   ///    51.317648888889 to 51.330925972223 /// 12.373587777778 to 12.373587777778
2
22.266372870738927   ///    51.330925972223 to 51.330725833334 /// 12.331557048612 to 12.331557048612
3
15.58337020224446   ///    51.330725833334 to 51.330585763889 /// 12.3313571875 to 12.3313571875
4
752.5837209936035   ///    51.330585763889 to 51.337350277778 /// 12.330539600695 to 12.330539600695
5
750.9689841929722   ///    51.337350277778 to 51.330600277777 /// 12.378552777778 to 12.378552777778
6
8.637683696038396   ///    51.330600277777 to 51.330677916667 /// 12.331373689236 to 12.331373689236
7
1623.3920079616098   ///    51.330677916667 to 51.345269582586 /// 12.330892222222 to 12.330892222222
8
1893.7012180701074   ///    51.345269582586 to 51.328248270942 /// 12.421201915266 to 12.421201915266
9
227.20340620158055   ///    51.328248270942 to 51.326206076389 /// 12.3418702103 to 12.3418702103
10
812.7186113150486   ///    51.326206076389 to 51.333511111111 /// 12.3403

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to heatmap_test.html!


In [69]:
myDfs[73817].sort_values(by=['start_time'], inplace=True)
print(myDfs[73817])

     bike_id  start_time    end_time  start_lat  start_lng    end_lat  \
0      73817  1546298062  1546298408  51.337350  12.378553  51.332373   
1      73817  1546301085  1546301779  51.332373  12.373868  51.323619   
2      73817  1546421072  1546422330  51.332176  12.405357  51.324615   
3      73817  1546457955  1546510170  51.324615  12.336922  51.324615   
4      73817  1546514580  1546514855  51.324615  12.336922  51.323013   
5      73817  1546516508  1546516787  51.323013  12.343557  51.324615   
6      73817  1546540523  1546541484  51.324615  12.336922  51.329400   
7      73817  1546549701  1546549764  51.329400  12.373656  51.329760   
8      73817  1546596561  1546597345  51.329760  12.373383  51.324615   
9      73817  1546621936  1546622754  51.324615  12.336922  51.330453   
10     73817  1546717250  1546718645  51.330298  12.373798  51.326127   
11     73817  1546734688  1546794441  51.325071  12.323580   0.000000   
12     73817  1546794448  1546794907   0.000000   0

In [73]:
print(myDfs[73817]['start_time'][1])

1546301085


In [63]:
myDfs[73817].to_csv("bikemyDfs.csv",index=False)

In [66]:
myDfs[73817].sort_values(by=['start_time'], inplace=True)
for ind in myDfs[73817].index[1:]:
    print(ind)
    print(str(df['end_time'][ind-1]))
    print(str(df['start_time'][ind]))
    print(df['end_time'][ind-1] < df['start_time'][ind])
    print("------------------")

1
1546298089
1546298045
False
------------------
2
1546300033
1546298059
False
------------------
3
1546300116
1546298060
False
------------------
4
1546298402
1546298062
False
------------------
5
1546298408
1546298073
False
------------------
6
1546300076
1546298184
False
------------------
7
1546300026
1546298273
False
------------------
8
1546298794
1546298391
False
------------------
9
1546298858
1546298450
False
------------------
10
1546299349
1546298520
False
------------------
11
1546298826
1546298590
False
------------------
12
1546299817
1546298625
False
------------------
13
1546299830
1546298838
False
------------------
14
1546299779
1546298955
False
------------------
15
1546301171
1546299244
False
------------------
16
1546300262
1546299465
False
------------------
17
1546299958
1546299487
False
------------------
18
1546300943
1546299496
False
------------------
19
1546299949
1546299995
True
------------------
20
1546301723
1546300251
False
------------------
21
1546300

1546363463
False
------------------
327
1546365722
1546363561
False
------------------
328
1546364121
1546364608
True
------------------
329
1546365695
1546364686
False
------------------
330
1546364827
1546365089
True
------------------
331
1546365550
1546365273
False
------------------
332
1546365967
1546365563
False
------------------
333
1546365701
1546365647
False
------------------
334
1546367204
1546365661
False
------------------
335
1546366800
1546365671
False
------------------
336
1546366808
1546365683
False
------------------
337
1546367178
1546366600
False
------------------
338
1546366632
1546366921
True
------------------
339
1546367219
1546368057
True
------------------
340
1546369674
1546368619
False
------------------
341
1546369207
1546368707
False
------------------
342
1546369047
1546368850
False
------------------
343
1546369824
1546369889
True
------------------
344
1546370131
1546370461
True
------------------
345
1546371110
1546370889
False
------------------
3

In [33]:
heatmap2 = KeplerGl()
heatmap2.add_data(data=df_points, name='points')
heatmap2

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'points': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, …

In [ ]:
# Save Heatmap
heatmap2.save_to_html(file_name='heatmap_hexagons1.html')